In [95]:
import traceback
import numpy as np
import nlopt
import math
import random
from scipy.linalg import block_diag, expm
from numpy import *

In [96]:
def symplectic_values(Z):
    """Return the symplectic eigenvalues of Z
    Parameters:
        Z a numpy matrix
    Returns:
        a list of symplectic eigenvalues"""

    size = len(Z)

    A = np.random.randn(size, size)
    A_symmetric = (A + A.T) / 2
    H = Z.T @ A_symmetric
    S = expm(H)

    result = S @ Z @ S.T

    values = np.diag(result)[::2]

    values_sum = np.sum(values)
    return values_sum

# omega = np.matrix([[0,0,1,0],
#                   [0,0,0,1],
#                   [-1,0,0,0],
#                   [0,-1,0,0]])
# omega = np.matrix([[0,1],
#                   [1,0]])
# a, b, c =symplectic_values(omega)
# print(symplectic_values(omega))

In [97]:
def measurement(theta, psi, phi):
    """Return the measurement for specific angles

    Parameters:
        theta: theta angle in [0,pi]
        psi: psi angle in [0,pi]
        z: phi angle in [0,2pi]
    Returns:
        A 4x4 numpy array
    """
    u=np.matrix([math.cos(theta)*math.cos(psi-phi), math.cos(theta)*math.sin(psi-phi), math.sin(theta)*math.cos(psi), math.sin(theta)*math.sin(psi)])
    v=np.matrix([math.cos(theta)*math.cos(psi-phi),math.cos(theta)*math.sin(psi-phi),math.sin(theta)*math.cos(psi),math.sin(theta)*math.sin(psi)])
    return np.outer(u,v.T)

In [98]:
def qmult_unit(n):
    """Generates a Haar-random unitary matrix of size n"""
    F = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(F)
    return Q @ np.diag(np.exp(1j * np.random.rand(n) * 2 * np.pi))

def symp_orth(n):
    """Generates a symplectic orthogonal matrix of size n"""
    B = qmult_unit(n)
    Re = np.real(B)
    Im = np.imag(B)
    return np.block([[Re, -Im],
                    [Im,  Re]])

In [99]:
def rand_rsymp(n,c):
    """Generate random symplectic matrix with specified symplectic eigenvalues.
    Parameters:
        n: dimension (2nx2n)
        c: symplectic eigenvalue(s) - either a scalar or n-vector
    Returns:
        A symplectic matrix
    """
    assert (len(c)==1 or len(c)==n),  (f"Expected vector component length for "
                                       f"symplectic matrix generation either 1 or {n}, got: {len(c)}")
    for scalar in c:
        assert scalar >= 1, f"Expected vector component for symplectic matrix generation >=, got: {scalar}"

    s = np.zeros(2*n)
    if(len(c)==1):
        s[0]=np.sqrt(c[0])
        s[1:n] = np.sort(1 + (s[0] - 1) * np.random.rand(n-1))[::-1]
        s[n:] = 1.0 / s[:n]
    else:
        c = np.sort(c)
        s[0:n] = c[::-1]
        s[n:2*n] = 1.0 / s[0:n]

    U=symp_orth(n)
    V=symp_orth(n)

    return U @ np.diag(s) @ V

In [100]:
def randCM(entg = 1):
    """Searches for a random state with a specific entanglement level
    Parameters:
        entg: entanglement level
    Returns:
        A 4x4 matrix
    """
    int = 5
    rot = 10
    num = 300000

    P = np.matrix([[1,0,0,0],
                  [0,0,1,0],
                  [0,1,0,0],
                  [0,0,0,1]])

    x = 1
    y = 1.1

    a=1
    b=10

    for i in range(num):
        # generate thermal state CM
        g1 = np.matrix([[random.uniform(x,y),0],
                        [0,random.uniform(x,y)]]),
        g2 = block_diag(g1, g1)


        # generate random symplectic transformations with symplectic eigenvalues between 1 and 7.5
        S=rand_rsymp(2,(random.uniform(a,b),random.uniform(a,b)))
        # apply symplectic transformations to obtain a general CM

        g3 = S.transpose()@g2@S
        cm = P.transpose()@g3@P

        # calculate the logarithmic negativity
        det_11 = np.linalg.det(cm[0:2, 0:2])
        det_22 = np.linalg.det(cm[2:4, 2:4])
        det_12 = np.linalg.det(cm[0:2, 2:4])
        det_cm = np.linalg.det(cm)

        f = (0.5 * (det_11 + det_22) - det_12
             - np.sqrt((0.5 * (det_11 + det_22) - det_12)**2 - det_cm))

        EN = -0.5 * np.log2(f)
        EN1 = np.round(EN * rot) / rot

        if EN1 == entg/int:
            return cm
    return None

In [101]:
def check_constraints(w_opt, M_list, min_val, num_ops=14, verbose = False):
    """
    Sanity check: verify all constraints are satisfied

    Returns:
        dict with constraint values and whether they're satisfied
    """
    W = np.sum([w_opt[k] * M_list[k] for k in range(num_ops)], axis=0)

    # Check 1: W ≥ 0 (PSD)
    eigvals = np.linalg.eigvalsh(W)
    min_eigval = np.min(np.real(eigvals))
    W_psd = min_eigval >= -1e-6  # Small tolerance for numerical errors

    # Check 2: sTr(Z11) + sTr(Z22) ≥ 0.5
    Z11 = W[0:2, 0:2]
    Z22 = W[2:4, 2:4]

    sTr_Z11 = symplectic_values(Z11)
    sTr_Z22 = symplectic_values(Z22)


    sTr_sum = sTr_Z11 + sTr_Z22
    sTr_satisfied = sTr_sum >= 0.5


    results = {
        'min_eigval_W': min_eigval,
        'W_is_PSD': W_psd,
        'sTr_sum': sTr_sum,
        'sTr_satisfied': sTr_satisfied,
        'objective_value': min_val,
        'all_constraints_ok': W_psd and sTr_satisfied
    }

    if verbose:
        print("=== Constraint Check ===")
        print(f"Objective (w·m): {min_val:.6f}")
        print(f"Min eigenvalue of W: {min_eigval:.6f} {'✓' if W_psd else '✗'}")
        print(f"det(Z11): {det_Z11:.6f}")
        print(f"det(Z22): {det_Z22:.6f}")
        if sTr_sum is not None:
            print(f"sTr(Z11) + sTr(Z22): {sTr_sum:.6f} {'✓' if sTr_satisfied else '✗'} (need ≥ 0.5)")
        else:
            print(f"sTr constraint: ✗ (negative determinants)")
        print(f"All constraints satisfied: {'YES ✓' if results['all_constraints_ok'] else 'NO ✗'}")

    return results

In [102]:
def steering_detection(M_list, m_list, num_ops=14):
    """
    Detect steering using NLopt with specific constraints

    Constraints:
    - W ≥ 0 (PSD)
    - sTr(Z11) + sTr(Z22) ≥ 0.5
    """
    n_vars = num_ops

    # Objective function
    def objective(w, grad):
        obj = np.dot(w, m_list)
        if grad.size > 0:
            grad[:] = m_list
        return float(obj)

    # Constraint 1: W ≥ 0 (PSD)
    def constraint_W_psd(w, grad):
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)
        eigvals = np.linalg.eigvalsh(W)
        min_eigval = np.min(np.real(eigvals))

        if grad.size > 0:
            grad[:] = 0
        # print(f'cond1:{min_eigval}')
        return float(min_eigval)

    # Constraint 2: sTr(Z11) + sTr(Z22) ≥ 0.5 (with MIN_DET check)
    def constraint_symplectic_trace(w, grad):
        MIN_DET = 1e-6  # Minimum determinant threshold
        W = np.sum([w[k] * M_list[k] for k in range(num_ops)], axis=0)
        Z11 = W[0:2, 0:2]
        Z22 = W[2:4, 2:4]

        sTr_Z11 = symplectic_values(Z11)
        sTr_Z22 = symplectic_values(Z22)
        constraint_val = sTr_Z11 + sTr_Z22 - 0.5

        if grad.size > 0:
            grad[:] = 0
        # print(f'cond4:{constraint_val}')
        return float(constraint_val)

    # Set up optimizer
    # opt = nlopt.opt(nlopt.LN_COBYLA, n_vars)
    # opt = nlopt.opt(nlopt.LD_SLSQP, n_vars)
    opt = nlopt.opt(nlopt.LN_AUGLAG, n_vars)
    # opt.set_local_optimizer(nlopt.opt(nlopt.LN_COBYLA, n_vars))
    # opt = nlopt.opt(nlopt.GN_ISRES, n_vars)


    opt.set_min_objective(objective)

    # Add all constraints
    opt.add_inequality_constraint(constraint_W_psd, 1e-6)
    opt.add_inequality_constraint(constraint_symplectic_trace, 1e-6)

    # Set bounds
    opt.set_lower_bounds(-10 * np.ones(n_vars))
    opt.set_upper_bounds(10 * np.ones(n_vars))

    # Set tolerances
    opt.set_xtol_rel(1e-6)
    opt.set_maxeval(10000)

    # Initial guess
    w0 = np.random.randn(n_vars)

    # Optimize
    try:
        w_opt = opt.optimize(w0)
        min_value = opt.last_optimum_value()
        return min_value, w_opt
    except nlopt.RoundoffLimited:
        min_value = opt.last_optimum_value()
        w_opt = w0  # Return last attempt
        print(f"  Roundoff limited, returning best found: {min_value:.6f}")
        return min_value, w_opt
    except np.linalg.LinAlgError as e:
        print(f"  LinAlgError during optimization: {e}")
        return np.inf, np.zeros(n_vars)
    except Exception as e:
        print(f"  Optimization failed")
        traceback.print_exc()
        return np.inf, None


In [103]:
# Main detection loop
num_ops = 14
num_rep = 5
num_ent = 1

nn = np.zeros((num_ops, num_ent), dtype=int)

for kdx in range(1, num_ent + 1):
    n = np.zeros(num_ops, dtype=int)

    # for jdx in range(num_rep):
        # Generate state
    while(True):
        g = randCM(kdx)

        # Generate random measurements
        theta = np.pi * np.random.rand(num_ops)
        psi = np.pi * np.random.rand(num_ops)
        phi = 2 * np.pi * np.random.rand(num_ops)

        M_list = []
        m_list = []

        for idx in range(num_ops):
            M = measurement(theta[idx], psi[idx], phi[idx])
            M_list.append(M)
            m_list.append(np.real(np.trace(M @ g)))

        # Run optimization
        min_val, w_opt = steering_detection(M_list, m_list, num_ops)
        results = check_constraints(w_opt, M_list, min_val, num_ops, False)

        # print(f"kdx={kdx}, jdx={jdx}, min_val={min_val:.6f}")

        # Check if steering detected
        if min_val < 1.0 and results['all_constraints_ok']:
            print(f"  → Steering detected! ")
            results = check_constraints(w_opt, M_list, min_val, num_ops, True)
            break
        elif min_val < 1.0:
            print(f"  → Nope {min_val}")
        else:
            print(f"  → No steering detected {min_val}")



  → Nope -172.13461261627276
  → Nope -4480.527653465537
  → Nope -1928.9563506110521
  → Nope -3506.2736537753253
  LinAlgError during optimization: Eigenvalues did not converge
  → No steering detected inf
  → Nope -2000.021294672196
  → Nope -236.2504769047316
  → Nope -1035.0272816559084
  → Nope -6510.933774976144
  → Nope -466.5575829241211
  → Nope -4115.076040383653
  → Nope -1778.5177582190754
  → Nope -1702.995774830772
  → Nope -353.4416523691075
  → Nope -349.4882319155581
  → No steering detected 4.181141393946268
  → Nope -173.9417351518176
  → Nope -155.36693761000228
  → Nope -3799.0315790594964
  → Nope -471.5659348672706
  → Nope -270.19269046611953
  → No steering detected 5.141239429580275
  → Nope -1768.7745959925057
  → Nope -190.47942136481623
  → Nope -758.6207667134966
  → Nope -313.6562381496169
  → Nope -567.0494176195015
  → Nope -458.20896287187577
  → Nope -563.2077122995622
  → Nope -1302.14849457505
  → Nope -3050.252023256075
  → Nope -512.4664708684588

KeyboardInterrupt: 